In [ ]:
%run Imports.ipynb
from quantpy.tomography.interval import ConfidenceInterval, _pop_hidden_keys, Mode

import numpy as np
import scipy.linalg as la
import scipy.stats as sts
import math

from enum import Enum, auto
from abc import ABC, abstractmethod
from scipy.interpolate import interp1d
from collections import Counter, defaultdict
from functools import partial
from cvxopt import matrix, solvers
from einops import repeat, rearrange
from matplotlib.lines import Line2D

from quantpy.geometry import hs_dst, trace_dst, if_dst
from quantpy.qobj import Qobj
from quantpy.routines import (
    _left_inv, _vec2mat, _mat2vec,
    _matrix_to_real_tril_vec, _real_tril_vec_to_matrix,
)
from quantpy.measurements import generate_measurement_matrix
from quantpy.tomography.polytopes.utils import count_confidence, count_delta
from quantpy.tomography.interval import PolytopeStateInterval, PolytopeProcessInterval

## Example of fidelity interval

### State

In [ ]:
n = 2
target_state = qp.qobj.GHZ(n)
dep_channel = qp.channel.depolarizing(0.1, n)
state = dep_channel.transform(target_state)

tmg = qp.StateTomograph(state)
tmg.experiment(10000)
state_hat = tmg.point_estimate()
fidelity = 1 - qp.if_dst(target_state, state)
interval = PolytopeStateInterval(tmg, 100, target_state=target_state)
(dist_min, dist_max), conf_levels = interval()
epsilons = 1 - conf_levels

In [ ]:
plt.figure(figsize=(20, 13), dpi=200)
plt.xlabel('$\\epsilon$')
plt.ylabel('Fidelity')
plt.grid()
plt.fill_between(epsilons, dist_min, dist_max, alpha=0.3)
plt.xscale('log')
plt.plot([epsilons[0], epsilons[-1]], [fidelity] * 2, '--')

### Process

In [ ]:
target_channel = qp.channel.depolarizing(0, 1)
channel = qp.channel.depolarizing(0.1, 1)

dim = 4 ** channel.n_qubits
tmg = qp.ProcessTomograph(channel, input_states='sic')
tmg.experiment(10000)
channel_hat = tmg.point_estimate()
fidelity = np.dot(target_channel.choi.bloch, channel.choi.bloch)
interval = PolytopeProcessInterval(tmg, target_channel=target_channel)
(dist_min, dist_max), conf_levels = interval()
epsilons = 1 - conf_levels

In [ ]:
plt.figure(figsize=(20, 13), dpi=200)
plt.xlabel('$\\epsilon$')
plt.ylabel('Fidelity')
plt.grid()
plt.fill_between(epsilons, dist_min, dist_max, alpha=0.3)
plt.xscale('log')
plt.plot([epsilons[0], epsilons[-1]], [fidelity] * 2, '--')

## Multiple intervals for fidelity

In [ ]:
target_channel = qp.channel.depolarizing(0, 1)
channel = qp.channel.depolarizing(0.1, 1)
true_fidelity = np.dot(target_channel.choi.bloch, channel.choi.bloch)
EPS = 1e-15

dim = 4 ** channel.n_qubits
n_meas_list = np.logspace(3, 5, 3)
min_results = []
max_results = []
for n_measurements in n_meas_list:
    n_measurements = int(n_measurements)
    tmg = qp.ProcessTomograph(channel, input_states='sic')
    tmg.experiment(n_measurements)
    bloch_indices = [i for i in range(dim ** 2) if i % dim != 0]

    povm_matrix = tmg.tomographs[0].povm_matrix
    n_measurements = tmg.tomographs[0].n_measurements

    meas_matrix = (np.reshape(povm_matrix * n_measurements[:, None, None] / np.sum(n_measurements),
                              (-1, povm_matrix.shape[-1])) 
                   * povm_matrix.shape[0])
    states_matrix = np.asarray([rho.T.bloch for rho in tmg.input_basis.elements])
    channel_matrix = np.einsum("i a, j b -> i j a b", states_matrix, meas_matrix[:, 1:]) * dim
    channel_matrix = rearrange(channel_matrix, "i j a b -> (i j) (a b)")
    A = np.ascontiguousarray(channel_matrix)

    n_trials = 100
    epsilons = list(np.logspace(-5, -0.2, 20)) + list(np.linspace(0.65, 0.99, 10))

    min_fidelity = defaultdict(list)
    max_fidelity = defaultdict(list)
    for _ in tqdm(range(n_trials)):
        tmg = qp.ProcessTomograph(channel, input_states='sic')
        tmg.experiment(n_measurements)
        frequencies = np.asarray([
            np.clip(ptmg.raw_results / ptmg.n_measurements[:, None], EPS, 1 - EPS)
            for ptmg in tmg.tomographs
        ])
        for i, epsilon in enumerate(epsilons):
            delta = count_delta(1 - epsilon, frequencies, tmg.tomographs[0].n_measurements)
            b = (np.hstack(np.concatenate(frequencies, axis=0)) + delta 
                 - repeat(meas_matrix[:, 0], 'a -> (a b)', b=len(states_matrix)))
            c = matrix(target_channel.choi.bloch[bloch_indices])
            G, h = matrix(A), matrix(b)
            sol = solvers.lp(c, G, h)
            min_fidelity[epsilon].append(1/dim + sol['primal objective'])
            sol = solvers.lp(-c, G, h)
            max_fidelity[epsilon].append(1/dim - sol['primal objective'])
    min_results.append(min_fidelity)
    max_results.append(max_fidelity)

In [ ]:
legend_elements = [
    Line2D([0], [0], color='b', lw=2, label='$n=10^3$'),
    Line2D([0], [0], color='g', lw=2, label='$n=10^4$'),
    Line2D([0], [0], color='r', lw=2, label='$n=10^5$'),
]

In [ ]:
fmts = ['b-', 'g-', 'r-']
plt.figure(figsize=(20, 13), dpi=200)
plt.grid()
plt.xlabel('$\\epsilon$')
plt.xscale('log')
plt.ylabel('Fidelity')
for min_fidelity, max_fidelity, fmt in zip(min_results, max_results, fmts):
    epsilons = sorted(list(min_fidelity.keys()))
    for i in range(n_trials):
        minfid = [min_fidelity[epsilon][i] for epsilon in epsilons]
        maxfid = [max_fidelity[epsilon][i] for epsilon in epsilons]
        plt.plot(epsilons, minfid, fmt, alpha=0.03)
        plt.plot(epsilons, maxfid, fmt, alpha=0.03)
    if fmt == 'b-':
        minfid = [min_fidelity[epsilon][-3] for epsilon in epsilons]  # -3, -6
        maxfid = [max_fidelity[epsilon][-3] for epsilon in epsilons]
        plt.plot(epsilons, minfid, fmt, epsilons, maxfid, fmt)
plt.legend(handles=legend_elements)
plt.plot([0, 1], [true_fidelity] * 2, '--k')
plt.figtext(0.02, 0.9, "a)")
plt.savefig('../imgs/fig2a.pdf')